# Testing the DEB model 2 (parameters estimation across substrates)
## Santruckova et al. (unpublished data)
1. Importing libraries

In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import dual_annealing
from scipy.optimize import differential_evolution

2. Defining DEB model

In [2]:
def DEBmodel (y, t, pars):
    #define initial pools
    S=y[0];    e=y[1];    X1=y[2];     CO2=y[3]
    #define parameters
    yA=pars[0]; 
    Km=pars[1];     
    v=pars[2];
    m=pars[3]; 
    g=pars[4]; 
    ce=pars[5];
    MX1=ce/4;
    #Define fluxes
    ##scaling function for substrate
    f=S/(Km+S)
    uptake=(v*ce/yA)*X1*f
    growth = (v*e-m*g)/(e+g)
    #Define derivatives
    dSdt = -uptake
    dedt = v*(f - e)
    dX1dt = X1*growth 
    dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v-growth)) - growth*X1*MX1
    return dSdt, dedt, dX1dt, dCO2dt;

3. Defining outputs from DEB model

In [3]:
def calcDEB (model, pars, t, y0):
    #model parameters
    ##yA, Km, v, m, g, ce
    pars_model=pars[0:6]
    #conversion factors
    ##ce, nX1
    conversions=pars[5:7]
    
    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))
    #calculate B, WallsProto,
    B=((conversions[0]/4 + conversions[0]*y[:, 1])*y[:, 2])
    WallsProto = (conversions[1]*conversions[0]/4)/(conversions[0]*y[:, 1])
    
    #Create data with predictions
    yhat = np.concatenate((y[:, 3].reshape(len(dSucrose.Time),1),#CO2
                           B.reshape(len(dSucrose.Time),1),
                           WallsProto.reshape(len(dSucrose.Time),1)), axis=1)
    
    return yhat

4. Defining objective function

In [4]:
def obj_funDEB (x):
    #define parameters
    ##yA_Sucrose, yA_SA, Km, v_Sucrose, v_SA, m, g, ce, nX1
    parsSucrose = x[np.array([0, 2, 3, 5, 6, 7, 8])]
    parsSA = x[np.array([1, 2, 4, 5, 6, 7, 8])]
        
    #initial conditions
    ##Sucrose
    S_iSucrose = dSucrose.Sinit[0]
    e_iSucrose =parsSucrose[6]/4/dSucrose.WP[0]
    X1_iSucrose = dSucrose.B[0]/(parsSucrose[5]*(0.25 + e_iSucrose))
    ##Sodium acetate
    S_iSA = dSA.Sinit[0]
    e_iSA =parsSucrose[6]/4/dSA.WP[0]
    X1_iSA = dSA.B[0]/(parsSucrose[5]*(0.25 + e_iSA))

    y0Sucrose = np.array([S_iSucrose, e_iSucrose, X1_iSucrose, 0])
    y0SA = np.array([S_iSA, e_iSA, X1_iSA, 0])
    
    #times
    t = dSucrose.Time
    #model simulations
    yhatSucrose = calcDEB(DEBmodel, parsSucrose, t, y0Sucrose)
    yhatSA = calcDEB(DEBmodel, parsSA, t, y0SA)
    yhat_full = np.concatenate((yhatSucrose, yhatSA))
    
    #observations
    obsSucrose=np.concatenate((np.array([dSucrose.Rc]).reshape(len(dSucrose.Time),1),
                           np.array([dSucrose.B]).reshape(len(dSucrose.Time),1),
                           np.array([dSucrose.WP]).reshape(len(dSucrose.Time),1)), axis=1)
    obsSA=np.concatenate((np.array([dSA.Rc]).reshape(len(dSucrose.Time),1),
                           np.array([dSA.B]).reshape(len(dSucrose.Time),1),
                           np.array([dSA.WP]).reshape(len(dSucrose.Time),1)), axis=1)
    obs=np.concatenate((obsSucrose, obsSA))
    #weights
    weightsSucrose=np.concatenate((np.nanmean(dSucrose.Rc).repeat(len(dSucrose.Time)).reshape(len(dSucrose.Time),1),
                            np.nanmean(dSucrose.B).repeat(len(dSucrose.Time)).reshape(len(dSucrose.Time),1),
                            np.nanmean((dSucrose.WP)).repeat(len(dSucrose.Time)).reshape(len(dSucrose.Time),1)),
                       axis=1)
    weightsSA=np.concatenate((np.nanmean(dSA.Rc).repeat(len(dSucrose.Time)).reshape(len(dSucrose.Time),1),
                            np.nanmean(dSA.B).repeat(len(dSucrose.Time)).reshape(len(dSucrose.Time),1),
                            np.nanmean((dSA.WP)).repeat(len(dSucrose.Time)).reshape(len(dSucrose.Time),1)),
                       axis=1)
    weights = np.concatenate((weightsSucrose, weightsSA))
    
    out=np.nansum(((yhat_full-obs)/weights)**2)
    return out

5. Calculate the goodness of fit

In [5]:
def goodnessDEB (x):
    #define parameters
    ##yA_Sucrose, yA_SA, Km, v_Sucrose, v_SA, m, g, ce, nX1
    parsSucrose = x[np.array([0, 2, 3, 5, 6, 7, 8])]
    parsSA = x[np.array([1, 2, 4, 5, 6, 7, 8])]
    
    #initial conditions
    ##Sucrose
    S_iSucrose = dSucrose.Sinit[0]
    e_iSucrose =parsSucrose[6]/4/dSucrose.WP[0]
    X1_iSucrose = dSucrose.B[0]/(parsSucrose[5]*(0.25 + e_iSucrose))
    ##Sodium acetate
    S_iSA = dSA.Sinit[0]
    e_iSA =parsSucrose[6]/4/dSA.WP[0]
    X1_iSA = dSA.B[0]/(parsSucrose[5]*(0.25 + e_iSA))

    y0Sucrose = np.array([S_iSucrose, e_iSucrose, X1_iSucrose, 0])
    y0SA = np.array([S_iSA, e_iSA, X1_iSA, 0])
    
    #times
    t = dSucrose.Time
    #model simulations
    yhatSucrose = calcDEB(DEBmodel, parsSucrose, t, y0Sucrose)
    yhatSA = calcDEB(DEBmodel, parsSA, t, y0SA)
    yhat_full = np.concatenate((yhatSucrose, yhatSA))
    
    #observations
    obsSucrose=np.concatenate((np.array([dSucrose.Rc]).reshape(len(dSucrose.Time),1),
                           np.array([dSucrose.B]).reshape(len(dSucrose.Time),1),
                           np.array([dSucrose.WP]).reshape(len(dSucrose.Time),1)), axis=1)
    obsSA=np.concatenate((np.array([dSA.Rc]).reshape(len(dSucrose.Time),1),
                           np.array([dSA.B]).reshape(len(dSucrose.Time),1),
                           np.array([dSA.WP]).reshape(len(dSucrose.Time),1)), axis=1)
    obs=np.concatenate((obsSucrose, obsSA))
    
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(x)*2 - 2*ll
    out = np.array([R2, ll, AIC])
    return out

Following function return the model solution for visualization in R

In [6]:
def predDEB (model, pars, t, y0):
    #model parameters
    pars_model=pars[0:6]
    #conversion factors
    conversions=pars[5:7]
    
    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))
    #calculate B, WallsProto,
    B=((conversions[0]/4 + conversions[0]*y[:, 1])*y[:, 2])
    WallsProto = (conversions[1]*conversions[0]/4)/(conversions[0]*y[:, 1])
    #Create data with predictions
    yhat = np.concatenate((y[:, 3].reshape(126,1),#CO2
                           B.reshape(126,1),
                           WallsProto.reshape(126,1)), axis=1)
    
    return yhat

6a. Reading data - Viden

In [7]:
dSucrose = pd.read_csv('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/SoilMBVariabilityData/VidenSucrose.csv', sep=',')
dSA = pd.read_csv('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/SoilMBVariabilityData/VidenSodiumAcetate.csv', sep=',')
print(dSucrose)
print(dSA)

   Unnamed: 0   Time         Rc         B        WP  Sinit
0           1    0.1   0.110000  2.130000  0.054500  51.29
1           2   24.0  15.831667  3.590000  0.359000  51.29
2           3   48.0  22.078333  3.723333  0.718833  51.29
3           4   72.0  24.503333  3.223333  0.624000  51.29
4           5   96.0  26.570000  2.593333  1.051833  51.29
5           6  120.0  27.845000  1.733333  0.496000  51.29
   Unnamed: 0   Time         Rc         B        WP  Sinit
0           1    0.1   0.166667  1.313333  0.054500  52.61
1           2   24.0   5.200000  1.826667  0.178167  52.61
2           3   48.0  10.148333  1.935000  0.147667  52.61
3           4   72.0  11.787500  2.020000  0.311000  52.61
4           5   96.0  12.358333  1.246667  0.173667  52.61
5           6  120.0  14.020000  2.445000  0.098167  52.61


7a. Estimating parameters - Viden

In [8]:
VidenDA=dual_annealing(obj_funDEB, [(0.05, 1), (0.05, 1), #yA_Sucrose, yA_SA
                                           (10, 1000),
                                           (0.001, 5), (0.001, 5), #v_Sucrose, v_SA
                                           (1e-12, 0.1), (0.1, 3), (0.1, 10), (0, 1)])

<ipython-input-3-f9ea69415c86>:13: RuntimeWarning: invalid value encountered in true_divide
  WallsProto = (conversions[1]*conversions[0]/4)/(conversions[0]*y[:, 1])


In [9]:
print(VidenDA)
print(goodnessDEB(VidenDA.x))

     fun: 4.6980290735575085
 message: ['Maximum number of iteration reached']
    nfev: 31441
    nhev: 0
     nit: 1000
    njev: 1344
  status: 0
 success: True
       x: array([3.14421630e-01, 4.62321050e-01, 8.43728097e+02, 3.04465214e-01,
       2.17025815e-01, 4.53432555e-02, 2.29302150e-01, 9.97334955e+00,
       4.10265643e-02])
[ 0.8642711  -2.44312019 22.88624038]


In [10]:
VidenDE=differential_evolution(obj_funDEB, [(0.05, 1), (0.05, 1), #yA_Sucrose, yA_SA
                                           (10, 1000),
                                           (0.001, 5), (0.001, 5), #v_Sucrose, v_SA
                                           (1e-12, 0.1), (0.1, 3), (0.1, 10), (0, 1)])

In [11]:
print(VidenDE)
print(goodnessDEB(VidenDE.x))

     fun: 4.648581754409326
     jac: array([-2.02504680e-05,  3.57047725e-05, -2.87236673e-04, -2.25597319e-05,
       -2.28261854e-04,  1.04076747e-03,  2.30571118e-04, -1.59872117e-06,
       -1.04423137e-03])
 message: 'Optimization terminated successfully.'
    nfev: 17250
     nit: 121
 success: True
       x: array([2.81736047e-01, 4.18366587e-01, 9.93895154e+02, 3.12603097e-01,
       2.22298158e-01, 4.36365989e-02, 2.09509160e-01, 2.21433518e+00,
       3.50359831e-02])
[ 0.86600732 -2.41186825 22.8237365 ]


8a. Solution

In [12]:
##Sucrose
S_iSucrose = dSucrose.Sinit[0]
e_iSucrose = VidenDE.x[8]/4/dSucrose.WP[0]
X1_iSucrose = dSucrose.B[0]/(VidenDE.x[7]*(0.25 + e_iSucrose))
##Sodium acetate
S_iSA = dSA.Sinit[0]
e_iSA = VidenDE.x[8]/4/dSA.WP[0]
X1_iSA = dSA.B[0]/(VidenDE.x[7]*(0.25 + e_iSA))

y0Sucrose = np.array([S_iSucrose, e_iSucrose, X1_iSucrose, 0])
y0SA = np.array([S_iSA, e_iSA, X1_iSA, 0])
    
#times
t = np.arange(126)
#model simulations
yhatSucrose = predDEB(DEBmodel, VidenDE.x[np.array([0, 2, 3, 5, 6, 7, 8])], t, y0Sucrose)
yhatSA = predDEB(DEBmodel, VidenDE.x[np.array([1, 2, 4, 5, 6, 7, 8])], t, y0SA)

VidenPred = np.concatenate((yhatSucrose, yhatSA))
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/VidenPred.csv', VidenPred, delimiter=",")

6b. Reading data - CB

In [13]:
dSucrose = pd.read_csv('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/SoilMBVariabilityData/CBSucrose.csv', sep=',')
dSA = pd.read_csv('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/SoilMBVariabilityData/CBAmonniumAcetate.csv', sep=',')
print(dSucrose)
print(dSA)

   Unnamed: 0   Time         Rc          B        WP  Sinit
0           1    0.1   0.106667  14.576667  2.270000  51.29
1           2   24.0   6.813333  45.540000  1.353333  51.29
2           3   48.0  20.923333  11.866667  3.146667  51.29
3           4   72.0  18.010000  12.840000  5.286667  51.29
4           5   96.0  18.853333  17.390000  2.793333  51.29
5           6  120.0  18.305000  13.113333  3.416667  51.29
   Unnamed: 0   Time         Rc          B        WP  Sinit
0           1    0.1   0.096667  14.240000  2.270000  51.28
1           2   24.0   2.720000  45.726667  1.871000  51.28
2           3   48.0   9.675000   7.910000  4.275000  51.28
3           4   72.0  10.980000  28.430000       NaN  51.28
4           5   96.0  10.263333  14.746667  4.013333  51.28
5           6  120.0   8.326667  20.703333  4.360000  51.28


7b. Estimating parameters - CB

In [ ]:
CBDA=dual_annealing(obj_funDEB, [(0.05, 1), (0.05, 1), #yA_Sucrose, yA_SA
                                           (10, 5000),
                                           (0.001, 5), (0.001, 5), #v_Sucrose, v_SA
                                           (1e-12, 0.1), (0.1, 3), (0.1, 10), (0, 1)])

In [ ]:
print(CBDA)
print(goodnessDEB(CBDA.x))

In [ ]:
CBDE=differential_evolution(obj_funDEB, [(0.05, 1), (0.05, 1), #yA_Sucrose, yA_SA
                                           (10, 5000),
                                           (0.001, 5), (0.001, 5), #v_Sucrose, v_SA
                                           (1e-12, 0.1), (0.1, 3), (0.1, 10), (0, 1)])

In [ ]:
print(CBDE)
print(goodnessDEB(CBDE.x))

In [ ]:
##Sucrose
S_iSucrose = dSucrose.Sinit[0]
e_iSucrose = CBDE.x[8]/4/dSucrose.WP[0]
X1_iSucrose = dSucrose.B[0]/(CBDE.x[7]*(0.25 + e_iSucrose))
##Sodium acetate
S_iSA = dSA.Sinit[0]
e_iSA = CBDE.x[8]/4/dSA.WP[0]
X1_iSA = dSA.B[0]/(CBDE.x[7]*(0.25 + e_iSA))

y0Sucrose = np.array([S_iSucrose, e_iSucrose, X1_iSucrose, 0])
y0SA = np.array([S_iSA, e_iSA, X1_iSA, 0])
    
#times
t = np.arange(126)
#model simulations
yhatSucrose = predDEB(DEBmodel, CBDE.x[np.array([0, 2, 3, 5, 6, 7, 8])], t, y0Sucrose)
yhatSA = predDEB(DEBmodel, CBDE.x[np.array([1, 2, 4, 5, 6, 7, 8])], t, y0SA)

CBPred = np.concatenate((yhatSucrose, yhatSA))
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/CBPred.csv', CBPred, delimiter=",")